In [1]:
import numpy as np, pandas as pd, scipy.stats as st
from scipy.stats import t, norm
from joblib import Parallel, delayed

In [2]:
data = pd.read_csv('data.csv')

num_obvs = 100_000
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
sector_indices = data['sector'].values
sec_loading = r[sector_indices]
datat = norm.ppf(data['p'])

In [3]:
def process_obs(obs):
    m_factor = obs[0]
    sec_factor = obs[:len(r)][sector_indices]
    res_factor = obs[len(r):]

    control_variate = (
            r[0]**0.5 * m_factor
            + (sec_loading - r[0])**0.5 * sec_factor
            + (1 - sec_loading)**0.5 * res_factor
    )

    ind = control_variate < datat
    loss = np.zeros(len(data))

    if np.any(ind):
        loss[ind] = data.loc[ind, 'm'].values + data.loc[ind, 'd'].values * np.clip(t.rvs(df=3, size=sum(ind)), -5, 5)

    return np.sum(loss), np.var(loss)

In [ ]:
from scipy.stats import genpareto

factors = np.random.normal(0, 1, (num_obvs, len(r) + len(data)))
antithetic_factors = -factors
combined_factors = np.vstack([factors, antithetic_factors])

answers = Parallel(n_jobs=-1, verbose=5)(delayed(process_obs)(obs) for obs in combined_factors)
answers = np.array(answers)

sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

threshold = np.percentile(sample_losses, 99)  # Top 1% of losses
print(f"Threshold for GPD: {threshold}")

excess_losses = sample_losses[sample_losses > threshold] - threshold

shape, loc, scale = genpareto.fit(excess_losses)

p_tail = 0.999  # Target quantile for VaR
n_excess = len(excess_losses)
VaR_parametric = threshold + scale / shape * ((num_obvs / n_excess * (1 - p_tail)) ** (-shape) - 1)